In [1]:
# Inspiration: https://towardsdatascience.com/hands-on-content-based-recommender-system-using-python-1d643bf314e4
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import libraries
import matplotlib.pyplot as plt
import numpy as np
from sentence_transformers import SentenceTransformer # required a pip install
import pandas as pd
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA
plt.style.use('ggplot')
plt.rcParams['font.family'] = 'sans-serif' 
plt.rcParams['font.serif'] = 'Ubuntu' 
plt.rcParams['font.monospace'] = 'Ubuntu Mono' 
plt.rcParams['font.size'] = 14 
plt.rcParams['axes.labelsize'] = 12 
plt.rcParams['axes.labelweight'] = 'bold' 
plt.rcParams['axes.titlesize'] = 12 
plt.rcParams['xtick.labelsize'] = 12 
plt.rcParams['ytick.labelsize'] = 12 
plt.rcParams['legend.fontsize'] = 12 
plt.rcParams['figure.titlesize'] = 12 
plt.rcParams['image.cmap'] = 'jet' 
plt.rcParams['image.interpolation'] = 'none' 
plt.rcParams['figure.figsize'] = (12, 10) 
plt.rcParams['axes.grid']=True
plt.rcParams['lines.linewidth'] = 2 
plt.rcParams['lines.markersize'] = 8
colors = ['xkcd:pale orange', 'xkcd:sea blue', 'xkcd:pale red', 'xkcd:sage green', 'xkcd:terra cotta', 'xkcd:dull purple', 'xkcd:teal', 'xkcd: goldenrod', 'xkcd:cadet blue',
'xkcd:scarlet']

/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Hello!


In [2]:
# Import dataset
data = pd.read_csv('imdb_top_1000.csv')
X = np.array(data.Overview)

# View the dataset
data = data[['Genre','Overview', 'Series_Title', 'Director']]
data.head()

,Genre,Overview,Series_Title,Director
0,Drama,Two imprisoned men bond over a number of years...,The Shawshank Redemption,Frank Darabont
1,"Crime, Drama",An organized crime dynasty's aging patriarch t...,The Godfather,Francis Ford Coppola
2,"Action, Crime, Drama",When the menace known as the Joker wreaks havo...,The Dark Knight,Christopher Nolan
3,"Crime, Drama",The early life and career of Vito Corleone in ...,The Godfather: Part II,Francis Ford Coppola
4,"Crime, Drama",A jury holdout attempts to prevent a miscarria...,12 Angry Men,Sidney Lumet


In [3]:
# Convert text to vector
text_data = X
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
embeddings = model.encode(text_data, show_progress_bar=True)

Batches: 100%|██████████| 32/32 [00:41<00:00,  1.28s/it]


In [4]:
# PCA (optional)
X = np.array(embeddings)
n_comp = 5
pca = PCA(n_components=n_comp)
pca.fit(X)
pca_data = pd.DataFrame(pca.transform(X))
pca_data.head()

,0,1,2,3,4
0,2.723328,1.605270,4.440936,-2.353442,2.101794
1,-0.132738,0.886310,4.694763,-2.714986,-1.788204
2,-5.688069,-1.777760,-0.752677,-0.699669,4.126630
3,-1.212180,4.458051,1.582215,-3.539145,-2.338916
4,-0.970992,-2.374561,3.174623,3.070695,-0.175007


In [6]:
# Compute cosine similarity and the function to generate recommendations

cos_sim_data = pd.DataFrame(cosine_similarity(X))
  

def give_recommendations(index,print_recommendation = False,print_recommendation_plots= False,print_genres =False):
  index_recomm =cos_sim_data.loc[index].sort_values(ascending=False).index.tolist()[1:6]
  movies_recomm =  data['Series_Title'].loc[index_recomm].values
  result = {'Movies':movies_recomm,'Index':index_recomm}
  if print_recommendation==True:
    print('The watched movie is this one: %s \n'%(data['Series_Title'].loc[index]))
    k=1
    for movie in movies_recomm:
      print('The number %i recommended movie is this one: %s \n'%(k,movie))
  if print_recommendation_plots==True:
    print('The plot of the watched movie is this one:\n %s \n'%(data['Overview'].loc[index]))
    k=1
    for q in range(len(movies_recomm)):
      plot_q = data['Overview'].loc[index_recomm[q]]
      print('The plot of the number %i recommended movie is this one:\n %s \n'%(k,plot_q))
      k=k+1
  if print_genres==True:
    print('The genres of the watched movie is this one:\n %s \n'%(data['Genre'].loc[index]))
    k=1
    for q in range(len(movies_recomm)):
      plot_q = data['Genre'].loc[index_recomm[q]]
      print('The plot of the number %i recommended movie is this one:\n %s \n'%(k,plot_q))
      k=k+1
  return result

In [9]:
give_recommendations(90,True)

The watched movie is this one: Incendies 

The number 1 recommended movie is this one: Amour 

The number 1 recommended movie is this one: Auf der anderen Seite 

The number 1 recommended movie is this one: Silver Linings Playbook 

The number 1 recommended movie is this one: Hannah and Her Sisters 

The number 1 recommended movie is this one: Trois couleurs: Blanc 



{'Movies': array(['Amour', 'Auf der anderen Seite', 'Silver Linings Playbook',
        'Hannah and Her Sisters', 'Trois couleurs: Blanc'], dtype=object),
 'Index': [483, 619, 752, 528, 967]}